# Examen 0101

## Parámetros

In [1]:
BASE_DIR = '/Users/efraflores/Desktop/EF/Diplo/data/other/DiegoF/01'
FILE_NAME = 'dataset_examen.csv'

## Funciones

### Column vs target

In [2]:
def col_vs_tgt(data, col, target='loan_status'):
    data['grouped'] = data[col]
    vs = data.pivot_table(index='grouped', columns=target, aggfunc={'id':'count'})
    vs = vs/vs.sum()
    return vs.style.format("{:.1%}").background_gradient('Blues')

### Group classes with low frequency

In [3]:
def group_lowfreq(df_col, p=0.05):
    tot_freq = df_col.value_counts(1).to_frame()
    tot_freq['group'] = [col if freq>p else 'other' for col,freq in zip(tot_freq.index,tot_freq.iloc[:,-1])]
    return df_col.map(tot_freq.iloc[:,-1].to_dict())

### Replace top words

In [4]:
from sklearn.feature_extraction.text import CountVectorizer

def replace_top_words(data, col, **kwargs):
    df = data.copy()
    cv = CountVectorizer(**kwargs)
    cv.fit(df[col])

    full = pd.DataFrame(cv.transform(df[col]).toarray(), index=zip(df.index,df[col].str.lower()), columns=cv.get_feature_names())
    full['total'] = full.sum(axis=1)

    replaced = []
    for i, tot in enumerate(full['total']):
        if tot==0: replaced.append('other')
        else:
            transposed = full.iloc[i,:-1].T.to_frame()
            unique_col = transposed.columns[0]
            filtered = transposed[transposed[unique_col]>0].sort_values(unique_col, ascending=False)
            replaced.append(filtered.index[0]) 
    
    return replaced, cv

## Importar

In [ ]:
import pandas as pd
from pathlib import Path

BASE_DIR = Path(BASE_DIR)
df = pd.read_csv(BASE_DIR.joinpath(FILE_NAME), low_memory=False)
print(df.shape)
df.head()

## Transformar

### Sólo dos estatus de préstamo

In [ ]:
df['loan_status'].value_counts()

In [ ]:
df = df[df['loan_status'].isin(['Fully Paid','Charged Off'])]
df['loan_status'].value_counts(1)

### Por variable

In [ ]:
vars_date = []
vars_cat = []
vars_bin = []
vars_num = []
df_replace = {}
df_bins = {}
count_v = {}

#### 1.	acc now delinq: El número de cuentas en las que el prestatario está ahora en mora

In [ ]:
df['acc_now_delinq'].value_counts(dropna=False)

In [ ]:
df['acc_now_delinq'] = (df['acc_now_delinq'] > 0)*1
df['acc_now_delinq'].value_counts(1, dropna=False)

In [ ]:
vars_bin.append('acc_now_delinq')
col_vs_tgt(df,'acc_now_delinq')

#### 2.	addr state: El estado proporcionado por el prestatario en la solicitud de préstamo

In [ ]:
df['addr_state'].value_counts(1, dropna=False)

In [ ]:
df['addr_state'] = group_lowfreq(df['addr_state'])
df['addr_state'].value_counts(1, dropna=False)

In [ ]:
vars_cat.append('addr_state')
col_vs_tgt(df,'addr_state')

#### 3.	all util: Límite de saldo a crédito en todas las operaciones

In [ ]:
df['all_util'].hist()

In [ ]:
df['all_util'].value_counts(1, dropna=False)

In [ ]:
df['all_util'] = (df['all_util'].notnull())*1
df['all_util'].value_counts(1, dropna=False)

In [ ]:
vars_bin.append('all_util')
col_vs_tgt(df, 'all_util')

#### 4.	annual inc: Los ingresos anuales autoinformadas proporcionados por el prestatario durante el registro

In [ ]:
df['annual_inc'].isnull().sum()

In [ ]:
df['annual_inc'].describe([i/10 for i in range(10)])

In [ ]:
df['annual_inc'], df_bins['annual_inc'] = pd.qcut(df['annual_inc'], q=5, retbins=True)
vars_cat.append('annual_inc')
col_vs_tgt(df, 'annual_inc')

#### 5.	annual inc joint: Los ingresos anuales autoinformadas combinados proporcionados por los co-prestatarios durante el registro

In [ ]:
df['annual_inc_joint'].isnull().mean()

In [ ]:
df.drop('annual_inc_joint', axis=1, inplace=True)

#### 6.	collection recovery fee: cargo posterior a la tarifa de cobro.


In [ ]:
df['collection_recovery_fee'].isnull().sum()

In [ ]:
df['collection_recovery_fee'] = (df['collection_recovery_fee']>0)*1
df['collection_recovery_fee'].value_counts(1, dropna=False)

In [ ]:
vars_bin.append('collection_recovery_fee')
col_vs_tgt(df, 'collection_recovery_fee')

#### 7.	collections 12 mths ex med: Número de colecciones en 12 meses, excluidas las colecciones médicas.


In [ ]:
df['collections_12_mths_ex_med'].value_counts(1, dropna=False)

In [ ]:
aux = df.copy()
aux['g'] = (aux['collections_12_mths_ex_med']>0)*1
col_vs_tgt(aux, 'g')

In [ ]:
df.drop(columns=['collections_12_mths_ex_med'], inplace=True)

#### 8.	delinq 2yrs: El número de más de 30 días de incidencias vencidas de morosidad en el archivo de crédito del prestatario durante los últimos 2 años.


In [ ]:
df['delinq_2yrs'].value_counts(1, dropna=False)

In [ ]:
df['delinq_2yrs'] = (df['delinq_2yrs']>0)*1
df['delinq_2yrs'].value_counts(1, dropna=False)

In [ ]:
vars_bin.append('delinq_2yrs')
col_vs_tgt(df, 'delinq_2yrs')

#### 9.	desc: Descripción del préstamo proporcionada por el prestatario.


In [ ]:
df['desc'].astype(str).str.split().str[0].value_counts(1, dropna=False).to_frame().head(22)

In [ ]:
df['desc'] = df['desc'].notnull()*1
df['desc'].value_counts(1, dropna=False)

In [ ]:
vars_bin.append('desc')
col_vs_tgt(df, 'desc')

#### 10.	dti: Una relación calculada utilizando los pagos mensuales totales de la deuda del prestatario sobre las obligaciones totales de la deuda, excluyendo la hipoteca y el préstamo LC solicitado, dividido por los ingresos mensuales autoinformadas del prestatario


In [ ]:
df['dti'].hist()

In [ ]:
vars_num.append('dti')

#### 11.	dti joint: Una relación calculada utilizando los pagos mensuales totales de los co-prestatarios sobre las obligaciones totales de deuda, excluyendo las hipotecas y el préstamo LC solicitado, dividido por los ingresos mensuales autoinformadas


In [ ]:
df['dti_joint'].value_counts(1, dropna=False)

In [ ]:
df.drop('dti_joint', axis=1, inplace=True)

#### 12.	earliest cr line: El mes en que se abrió la línea de crédito reportada más temprana del prestatario.


In [ ]:
df['earliest_cr_line'].value_counts(1, dropna=False)

In [ ]:
df['earliest_cr_line'] = pd.to_datetime(df['earliest_cr_line'], format=r'%b-%Y')
vars_date.append('earliest_cr_line')

#### 13.	emp length: Duración del empleo en años. Los valores posibles están entre 0 y 10, donde 0 significa menos de un año y 10 significa diez o más años.


In [ ]:
df['emp_length'].value_counts(0, dropna=False).sort_index()

In [ ]:
import numpy as np

df_replace['emp_length'] = {np.nan: '00-02',
                            '< 1 year': '00-02',
                            '1 year': '00-02',
                            '2 years': '00-02',
                            '3 years': '03-05',
                            '4 years': '03-05',
                            '5 years': '03-05',
                            '6 years': '06-08',
                            '7 years': '06-08',
                            '8 years': '06-08',
                            '9 years': '09-99',
                            '10+ years': '09-99'}

df['emp_length'] = df['emp_length'].map(df_replace['emp_length'])
df['emp_length'].value_counts(1, dropna=False)

In [ ]:
vars_cat.append('emp_length')
col_vs_tgt(df, 'emp_length')

#### 14.	emp title: El título del trabajo proporcionado por el Prestatario al solicitar el préstamo.*


In [ ]:
df['emp_title'].fillna('unknown', inplace=True)
df['emp_title'].value_counts(1, dropna=False)

In [ ]:
df['emp_title'], count_v['emp_title'] = replace_top_words(df, 'emp_title', max_features=10, stop_words=['and','of','us','inc','the','city','county','country','group','center'])
df['emp_title'].value_counts(1, dropna=False)

In [ ]:
vars_cat.append('emp_title')
col_vs_tgt(df, 'emp_title')

#### 15.	funded amnt: El importe total comprometido para ese préstamo en ese momento.


In [ ]:
df['funded_amnt'].isnull().sum()

In [ ]:
df['funded_amnt'].hist()

In [ ]:
vars_num.append('funded_amnt')

#### 16.	funded amnt inv: La cantidad total comprometida por los inversores para ese préstamo en ese momento.


In [ ]:
df['funded_amnt_inv'].isnull().sum()

In [ ]:
df[['funded_amnt_inv','funded_amnt']].corr()

In [ ]:
vars_num.append('funded_amnt_inv')

#### 17.	grade: LC asignó grado de préstamo.


In [ ]:
df['grade'].value_counts(1, dropna=False)

In [ ]:
vars_cat.append('grade')
col_vs_tgt(df, 'grade')

#### 18.	home ownership: El estado de propiedad de la vivienda proporcionado por el prestatario durante el registro. Nuestros valores son: ALQUILER, PROPIO, HIPOTECA, OTRO.


In [ ]:
df['home_ownership'].value_counts(1, dropna=False)

In [ ]:
df_replace['home_ownership'] = {'MORTGAGE': 'MORTGAGE',
                                'RENT': 'RENT',
                                'OWN': 'OTHER',
                                'OTHER': 'OTHER',
                                'NONE': 'OTHER',
                                'ANY': 'OTHER'}
                                
df['home_ownership'] = df['home_ownership'].map(df_replace['home_ownership'])
df['home_ownership'].value_counts(1, dropna=False)

In [ ]:
vars_cat.append('home_ownership')
col_vs_tgt(df, 'home_ownership')

#### 19.	id: Una LC única asignada ID para el listado de préstamos.


In [ ]:
df['id'].isnull().sum()

#### 20.	il util: Relación entre el saldo corriente total y el límite de crédito/crédito alto en todos los valores de la instalación.


In [ ]:
df['il_util'].value_counts(1, dropna=False)

In [ ]:
aux = df.copy()
aux['grouped'] = (df['il_util'].notnull())*1
col_vs_tgt(aux, 'grouped')

In [ ]:
df.drop('il_util', axis=1, inplace=True)

#### 21.	initial list status: El estado de listado inicial del préstamo. Los valores posibles son – W, F.


In [ ]:
df['initial_list_status'].value_counts(1, dropna=False)

In [ ]:
df_replace['initial_list_status'] = {'f':0, 'w':1}

df['initial_list_status'] = df['initial_list_status'].map(df_replace['initial_list_status'])

vars_bin.append('initial_list_status')
col_vs_tgt(df, 'initial_list_status')

#### 22.	inq fi: Número de consultas de finanzas personales.


In [ ]:
df['inq_fi'].value_counts(1, dropna=False)

In [ ]:
aux = df.copy()
aux['grouped'] = (df['inq_fi'].notnull())*1
col_vs_tgt(aux, 'grouped')

In [ ]:
df.drop('inq_fi', axis=1, inplace=True)

#### 23.	inq last 12m: Número de consultas de crédito en los últimos 12 meses.


In [ ]:
df['inq_last_12m'].value_counts(1, dropna=False)

In [ ]:
aux = df.copy()
aux['aux'] = (aux['inq_last_12m'].notnull())*1
col_vs_tgt(aux, 'aux')

In [ ]:
df.drop('inq_last_12m', axis=1, inplace=True)

#### 24.	inq last 6mths: El número de consultas en los últimos 6 meses (excluyendo las consultas de automóviles e hipotecas).


In [ ]:
df['inq_last_6mths'].value_counts(1, dropna=False)

In [ ]:
df_replace['inq_last_6mths'] = {x:str(x) if x<3 else '>2' for x in range(11)}

df['inq_last_6mths'] = df['inq_last_6mths'].map(df_replace['inq_last_6mths'])
df['inq_last_6mths'].value_counts(1, dropna=False)

In [ ]:
vars_cat.append('inq_last_6mths')
col_vs_tgt(df, 'inq_last_6mths')

#### 25.	installment: El pago mensual adeudado por el prestatario si el préstamo se origina.


In [ ]:
df['installment'].hist()

In [ ]:
vars_num.append('installment')

#### 26.	int rate: Tasa de interés del préstamo.


In [ ]:
df['int_rate'].hist()

In [ ]:
vars_num.append('int_rate')

#### 27.	issue d: El mes en que se financió el préstamo


In [ ]:
df['issue_d'].value_counts(1, dropna=False)

In [ ]:
df['issue_d'] = pd.to_datetime(df['issue_d'], format=r'%b-%Y')
vars_date.append('issue_d')

#### 28.	last credit pull d: El mes más reciente LC sacó crédito para este préstamo.


In [ ]:
df['last_credit_pull_d'].value_counts(1, dropna=False)

In [ ]:
df['last_credit_pull_d'] = pd.to_datetime(df['last_credit_pull_d'], format=r'%b-%Y')
vars_date.append('last_credit_pull_d')

#### 29.	last pymnt amnt: Último importe total de pago recibido.


In [ ]:
df['last_pymnt_amnt'].hist()

In [ ]:
vars_num.append('last_pymnt_amnt')

#### 30.	last pymnt d: El mes pasado se recibió el pago.


In [ ]:
df['last_pymnt_d'].value_counts(1, dropna=False)

In [ ]:
df['last_pymnt_d'] = pd.to_datetime(df['last_pymnt_d'], format=r'%b-%Y')
vars_date.append('last_pymnt_d')

#### 31.	loan amnt: El monto indicado del préstamo solicitado por el prestatario. Si en algún momento, el departamento de crédito reduce el monto del préstamo, entonces se reflejará en este valor.


In [ ]:
df['loan_amnt'].hist()

In [ ]:
vars_num.append('loan_amnt')

#### 32.	loan status: Estado actual del préstamo.


In [ ]:
df['loan_status'].value_counts(1, dropna=False)

#### 33.	max bal bc: Saldo corriente máximo adeudado en todas las cuentas rotativas.


In [ ]:
df['max_bal_bc'].value_counts(1, dropna=False)

In [ ]:
aux = df.copy()
aux['grouped'] = (df['max_bal_bc'].notnull())*1
col_vs_tgt(aux, 'grouped')

In [ ]:
df.drop('max_bal_bc', axis=1, inplace=True)

#### 34.	member id: Una LC única asignada id para el miembro prestatario.


In [ ]:
len(df) , len(df['member_id'].unique())

#### 35.	mths since last delinq: El número de meses desde la última morosidad del prestatario.


In [ ]:
df['mths_since_last_delinq'].value_counts(1, dropna=False)

In [ ]:
df['mths_since_last_delinq']= (df['mths_since_last_delinq'].notnull())*1
vars_bin.append('mths_since_last_delinq')
col_vs_tgt(df, 'mths_since_last_delinq')

#### 36.	mths since last major derog: Meses desde la calificación más reciente de 90 días o peor.


In [ ]:
df['mths_since_last_major_derog'].value_counts(1, dropna=False)

In [ ]:
df['mths_since_last_major_derog']= (df['mths_since_last_major_derog'].notnull())*1
vars_bin.append('mths_since_last_major_derog')
col_vs_tgt(df, 'mths_since_last_major_derog')

#### 37.	mths since last record: El número de meses transcurridos desde el último registro público.


In [ ]:
df['mths_since_last_record'].value_counts(1, dropna=False)

In [ ]:
df['mths_since_last_record']= (df['mths_since_last_record'].notnull())*1
vars_bin.append('mths_since_last_record')
col_vs_tgt(df, 'mths_since_last_record')

#### 38.	mths since rcnt il: Meses desde que se abrieron las cuentas a plazos más recientes.


In [ ]:
df['mths_since_rcnt_il'].value_counts(1, dropna=False)

In [ ]:
aux = df.copy()
aux['aux'] = (aux['mths_since_rcnt_il'].notnull())*1
col_vs_tgt(aux, 'aux')

In [ ]:
df.drop('mths_since_rcnt_il', axis=1, inplace=True)

#### 39.	next pymnt d: Próxima fecha de pago programada.


In [ ]:
df['next_pymnt_d'].value_counts(1, dropna=False)

In [ ]:
df.drop('next_pymnt_d', axis=1, inplace=True)

#### 40.	open acc: El número de líneas de crédito abiertas en el archivo de crédito del prestatario.


In [ ]:
df['open_acc'].hist()

In [ ]:
vars_num.append('open_acc')

#### 41.	open acc 6m: Número de operaciones abiertas en los últimos 6 meses


In [ ]:
df['open_acc'].hist()

In [ ]:
vars_num.append('open_acc')

#### 42.	open il 12m: Número de cuentas a plazos abiertas en los últimos 12 meses.


In [ ]:
df['open_il_12m'].isnull().mean()

In [ ]:
aux = df.copy()
aux['aux'] = (aux['open_il_12m'].notnull())*1
col_vs_tgt(aux, 'aux')

In [ ]:
df.drop('open_il_12m', axis=1, inplace=True)

#### 43.	open il 24m: Número de cuentas a plazos abiertas en los últimos 24 meses.


In [ ]:
df['open_il_24m'].value_counts(1, dropna=False)

In [ ]:
aux = df.copy()
aux['aux'] = (aux['open_il_24m'].notnull())*1
col_vs_tgt(aux, 'aux')

In [ ]:
df.drop('open_il_24m', axis=1, inplace=True)

#### 44.	open il 6m: Número de operaciones a plazos actualmente activas.


In [ ]:
df['open_il_6m'].isnull().mean()

In [ ]:
aux = df.copy()
aux['aux'] = (aux['open_il_6m'].notnull())*1
col_vs_tgt(aux, 'aux')

In [ ]:
df.drop('open_il_6m', axis=1, inplace=True)

#### 45.	open rv 12m: Número de operaciones rotativas abiertas en los últimos 12 meses.


In [ ]:
df['open_rv_12m'].isnull().mean()

In [ ]:
aux = df.copy()
aux['aux'] = (aux['open_rv_12m'].notnull())*1
col_vs_tgt(aux, 'aux')

In [ ]:
df.drop('open_rv_12m', axis=1, inplace=True)

#### 46.	open rv 24m: Número de operaciones rotativas abiertas en los últimos 24 meses.


In [ ]:
df['open_rv_24m'].isnull().mean()

In [ ]:
aux = df.copy()
aux['aux'] = (aux['open_rv_24m'].notnull())*1
col_vs_tgt(aux, 'aux')

In [ ]:
df.drop('open_rv_24m', axis=1, inplace=True)

#### 47.	out prncp: Capital pendiente restante para el monto total financiado.


In [ ]:
df['out_prncp'].value_counts(1, dropna=False)

In [ ]:
df.drop('out_prncp', axis=1, inplace=True)

#### 48.	out prncp inv: Capital pendiente restante por parte del importe total financiado por los inversores.


In [ ]:
df['out_prncp_inv'].value_counts(1, dropna=False)

In [ ]:
df.drop('out_prncp_inv', axis=1, inplace=True)

#### 49.	policy code: disponible públicamente policy code=1 nuevos productos no disponibles públicamente policy code=2.


In [ ]:
df['policy_code'].value_counts(1, dropna=False)

In [ ]:
df.drop('policy_code', axis=1, inplace=True)

#### 50.	pub rec: Número de registros públicos despectivos.


In [ ]:
df['pub_rec'].value_counts(1, dropna=False)

In [ ]:
df['pub_rec'] = (df['pub_rec']>0)*1
df['pub_rec'].value_counts(1, dropna=False)

In [ ]:
vars_bin.append('pub_rec')
col_vs_tgt(df, 'pub_rec')

#### 51.	purpose: Una categoría proporcionada por el prestatario para la solicitud de préstamo.


In [ ]:
df['purpose'].value_counts(1, dropna=False)

In [ ]:
df['purpose'] = group_lowfreq(df['purpose'], p=0.022)
df['purpose'].value_counts(1, dropna=False)

In [ ]:
vars_cat.append('purpose')
col_vs_tgt(df, 'purpose')

#### 52.	pymnt plan: Indica si se ha establecido un plan de pago para el préstamo.


In [ ]:
df['pymnt_plan'].value_counts(1, dropna=False)

In [ ]:
df.drop('pymnt_plan', axis=1, inplace=True)

#### 53.	recoveries: cargo posterior a la recuperación bruta.


In [ ]:
df['recoveries'].value_counts(1, dropna=False)

In [ ]:
df['recoveries'] = (df['recoveries']>0)*1
df['recoveries'].value_counts(1, dropna=False)

In [ ]:
vars_bin.append('recoveries')
col_vs_tgt(df, 'recoveries')

#### 54.	revol bal: Saldo rotatorio de crédito total.


In [ ]:
df['revol_bal'].describe([i/10 for i in range(10)])

In [ ]:
df['revol_bal'], df_bins['revol_bal'] = pd.qcut(df['revol_bal'], q=5, retbins=True)
vars_cat.append('revol_bal')
col_vs_tgt(df, 'revol_bal')

#### 55.	revol util: Tasa de utilización de la línea rotativa, o la cantidad de crédito que el prestatario está utilizando en relación con todo el crédito rotatorio disponible.


In [ ]:
df['revol_util'].isnull().mean()

In [ ]:
df.dropna(subset=['revol_util'], inplace=True)
df['revol_util'].describe([i/10 for i in range(10)])

In [ ]:
df['revol_util'], df_bins['revol_util'] = pd.qcut(df['revol_util'], q=5, retbins=True)
vars_cat.append('revol_util')
col_vs_tgt(df, 'revol_util')

#### 56.	sub grade: Subsuelo de préstamo asignado LC.


In [ ]:
df[['grade','sub_grade']].value_counts().to_frame().sort_index()

In [ ]:
vars_cat.append('sub_grade')
col_vs_tgt(df, 'sub_grade')

#### 57.	term: El número de pagos del préstamo. Los valores están en meses y pueden ser 36 o 60.


In [ ]:
df['term'].value_counts(1, dropna=False)

In [ ]:
df_replace['term'] = {' 36 months':0, ' 60 months':1}
df['term'] = df['term'].map(df_replace['term'])
df['term'].value_counts(1, dropna=False)

In [ ]:
vars_bin.append('term')
col_vs_tgt(df, 'term')

#### 58.	title: El título del préstamo proporcionado por el prestatario.


In [ ]:
df['title'].value_counts(1, dropna=False)

In [ ]:
df['title'].fillna('unknown', inplace=True)
df['title'], count_v['title'] = replace_top_words(df, 'title', max_features=10, stop_words=['and','of','us','the','to'])
df['title'].value_counts(1, dropna=False)

In [ ]:
df['title'].value_counts(1, dropna=False)

In [ ]:
df_replace['title'] = {'consolidation': 'consolidation',
                        'other': 'other',
                        'card': 'credit',
                        'loan': 'loan',
                        'home': 'home',
                        'debt': 'consolidation',
                        'credit': 'credit',
                        'payoff': 'consolidation',
                        'improvement': 'home',
                        'refinancing': 'loan'}
df['title'] = df['title'].map(df_replace['title'])
df['title'].value_counts(1, dropna=False)

In [ ]:
vars_cat.append('title')
col_vs_tgt(df, 'title')

#### 59.	tot coll amt: Montos totales de cobro adeudados.


In [ ]:
df['tot_coll_amt'].value_counts(1, dropna=False)

In [ ]:
import numpy as np

df_replace['tot_coll_amt'] = {np.nan:'unknown', 0:'00'}
df['tot_coll_amt'] = df['tot_coll_amt'].map(df_replace['tot_coll_amt']).fillna('>0')
df['tot_coll_amt'].value_counts(1, dropna=False)

In [ ]:
vars_cat.append('tot_coll_amt')
col_vs_tgt(df, 'tot_coll_amt')

#### 60.	tot cur bal: Saldo corriente total de todas las cuentas.


In [ ]:
df['tot_cur_bal'].value_counts(1, dropna=False)

In [ ]:
df['tot_cur_bal'] = (df['tot_cur_bal'].notnull())*1
vars_bin.append('tot_cur_bal')
col_vs_tgt(df, 'tot_cur_bal')

#### 61.	total bal il: Saldo actual total de todas las cuentas a plazos.


In [ ]:
df['total_bal_il'].value_counts(1, dropna=False)

In [ ]:
aux = df.copy()
aux['aux'] = (aux['total_bal_il'].notnull())*1
col_vs_tgt(aux, 'aux')

In [ ]:
df.drop('total_bal_il', axis=1, inplace=True)

#### 62.	total cu tl: Número de operaciones financieras.


In [ ]:
df['total_cu_tl'].value_counts(1, dropna=False)

In [ ]:
aux = df.copy()
aux['aux'] = (aux['total_cu_tl'].notnull())*1
col_vs_tgt(aux, 'aux')

In [ ]:
df.drop('total_cu_tl', axis=1, inplace=True)

#### 63.	total pymnt: Pagos recibidos hasta la fecha por el importe total financiado.


In [ ]:
df['total_pymnt'].hist()

In [ ]:
vars_num.append('total_pymnt')

#### 64.	total pymnt inv: Pagos recibidos hasta la fecha por una parte del importe total financiado por los inversores.


In [ ]:
df['total_pymnt_inv'].hist()

In [ ]:
df[['total_pymnt_inv','total_pymnt']].corr()

In [ ]:
vars_num.append('total_pymnt_inv')

#### 65.	total rec int: Intereses recibidos hasta la fecha.


In [ ]:
df['total_rec_int'].hist()

In [ ]:
df['total_rec_int'].describe()

In [ ]:
df['total_rec_int'], df_bins['total_rec_int'] = pd.qcut(df['total_rec_int'], q=5, retbins=True)
vars_cat.append('total_rec_int')
col_vs_tgt(df, 'total_rec_int')

#### 66.	total rec prncp: Principal recibido hasta la fecha.


In [ ]:
df['total_rec_prncp'].hist()

In [ ]:
vars_num.append('total_rec_int')

#### 67.	total rev hi lim: Límite de crédito/crédito revolving total alto.


In [ ]:
df['total_rev_hi_lim'].isnull().mean()

In [ ]:
df['total_rev_hi_lim'].describe([i/10 for i in range(10)])

In [ ]:
df['total_rev_hi_lim'], df_bins['total_rev_hi_lim'] = pd.qcut(df['total_rev_hi_lim'], q=5, retbins=True)
vars_cat.append('total_rev_hi_lim')
col_vs_tgt(df, 'total_rev_hi_lim')

#### 68.	url: Dirección URL de la página lc con datos de listado.


In [ ]:
df['url'].str.split('/').str[-1].str.split('=').str[-1].value_counts(dropna=False)

#### 69.	verification status: Indica si lc verificó los ingresos, no los verificó o si se verificó la fuente de ingresos.


In [ ]:
df['verification_status'].value_counts(1, dropna=False)

In [ ]:
vars_cat.append('verification_status')

#### 70.	verified status joint: Indica si los ingresos conjuntos de los co-prestatariosMfueron verificados por LC, no verificados, o si la fuente de ingresos fue verificada.


In [ ]:
df['verification_status_joint'].value_counts(1, dropna=False)

In [ ]:
df.drop('verification_status_joint', axis=1, inplace=True)

#### 71.	zip code: Los primeros 3 números del código postal proporcionado por el prestatario en la solicitud de préstamo

In [ ]:
df['zip_code'].value_counts(1, dropna=False).to_frame().sort_index()

In [ ]:
df['zip_code'] = df['zip_code'].str[:1].str.zfill(2)
df['zip_code'].value_counts(1, dropna=False)

In [ ]:
vars_cat.append('zip_code')
col_vs_tgt(df, 'zip_code')

## Ingeniería de variables

In [ ]:
df.to_csv(BASE_DIR.joinpath(f'checkpoint_{FILE_NAME}'), index=False)

# import pandas as pd
# from pathlib import Path
# BASE_DIR = Path(BASE_DIR)
# df = pd.read_csv(BASE_DIR.joinpath(f'checkpoint_{FILE_NAME}'), low_memory=False)

## Modelo

In [ ]:
df[vars_date+vars_bin+vars_cat+vars_num].sample()

### Pre-procesamiento

In [ ]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

X = df.iloc[:,:-1].copy()
y = df.iloc[:,-1].values

X_train, X_test, y_train, y_test = train_test_split(X,y,
                                                    train_size=0.77,
                                                    random_state=22)
mm_x = MinMaxScaler()

### Arquitectura

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression

linear_reg = LinearRegression()

model_reg = Pipeline(steps=[('scaler', mm_x),
                            ('model', linear_reg)])

print(f"Score: {'{:.2%}'.format(model_reg.fit(X_train,y_train).score(X_test,y_test))}")
print(f"Training score: {'{:.2%}'.format(model_reg.score(X_train,y_train))}")
print("\nThese are the most influential variables:")
coef = pd.DataFrame(zip(X.columns,model_reg[1].coef_)).sort_values(1,0,0).reset_index(drop=True)
coef.head().append(coef.tail())

## Predicción

In [ ]:
import numpy as np
val['2018_26'] = np.clip(model_reg.predict(val), 0, 1e10).round()
val.head()

In [ ]:
val[['2018_26']].to_csv(BASE_DIR.joinpath(f"Bren_{VALID_NAME_SUB}_final.csv"))